In [136]:
%%capture
%run full_spectrum.ipynb

In [137]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show, push_notebook, output_file, save, curdoc
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Dropdown, CheckboxGroup, Button, Paragraph
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.models.tools import BoxSelectTool, LassoSelectTool
from bokeh.layouts import row, column, gridplot, layout
from bokeh.models.formatters import BasicTickFormatter
from bokeh.models.tickers import AdaptiveTicker

tick_form = BasicTickFormatter()
tick_form.use_scientific
tick_form.precision

'auto'

In [138]:
data = dict(bokeh_data)

In [139]:
mean = data['Saturated']['sat_raw'].mean()
data['Saturated']['sat_raw'] -= mean
data['Saturated']['sat_background'] -= mean

In [140]:
# data['Broadened'].t /= 1000
theory_total = data['Broadened']['hyperfine_0'].copy()
for i in range(1,16):
    theory_total += data['Broadened']['hyperfine_{}'.format(i)]

data['Broadened']['Total'] = theory_total.copy()
data['FP'].V /= 2
data['Broadened']['photo'] = data['Broadened']['photo_raw']
data['Broadened']['photo_nb'] *= -1 
data['Broadened']['link'] = data['Broadened']['photo_raw']
data['Broadened']['link2'] = data['Broadened']['photo_raw']
data['Saturated'].t /= 1000
data['Saturated']['photo'] = data['Saturated']['sat_raw']

link_offset = float(np.array(data['Broadened']['link'][abs(data['Broadened']['t']) <0.001]) -\
        np.array(data['Saturated']['sat_background'][abs(data['Saturated']['t']-0) <0.0005]))
data['Broadened']['link']  -= link_offset*0.96
data['Broadened']['link2']  -= link_offset*0.99




In [141]:
#output_notebook()

In [142]:
# dropdown_update = CustomJS(args=dict( all_data_source = all_data_source, source = source, xaxis = xaxis,
#                                      dropdown = dropdown), code = """
#     all_data = all_data_source.get('data');
#     data = source.get('data');
    
#     if(dropdown.value == 't'){
#         data['x'] = all_data['x_t'];
#         data['x_fp'] = all_data['x_fp_t'];
#         xaxis.attributes.axis_label = 't[s]';
#     }else{
#         data['x'] = all_data['x'];
#         data['x_fp'] = all_data['x_fp'];
#         xaxis.attributes.axis_label = 'f[MHz]';
#     }
#     source.change.emit();
#      xaxis.trigger('change');

#     """)


In [150]:
p = figure(height = 300, width = 420, tools="pan,wheel_zoom,box_zoom,reset,box_select, lasso_select", 
          title = 'Spectrum without saturation beam') 
p.background_fill_color = 'black'
p.border_fill_color = 'lightgrey'
p.grid.grid_line_color = 'white'
p.grid.grid_line_alpha = .3
p.xaxis.axis_label = 'f[GHz]'
p.yaxis.axis_label = 'Intensity (arb. units)'


source = ColumnDataSource(data = data['Broadened'])
source_fp = ColumnDataSource(data = dict(t = data['FP'].t, V = data['FP'].V))
                          

photo = p.circle('t','photo', source = source, size = 1.0)

fp = p.line('t', 'V', source = source_fp, color = 'orange')

theory = []
for i in range(16):
    theory.append(p.line('t', 'hyperfine_{}'.format(i), source = source, color = 'red', line_dash= 'dashed'))
    theory[-1].visible = False

menu = [("Frequency [Ghz]", "freq"), ("Time [s]", "t")]
# dropdown = Dropdown(label="x-axis", button_type="default", menu=menu)

checkbox_group = CheckboxGroup(
        labels=["Photometer", "Fabry-perot", "Theory - Hyperfine", "Theory - Total"], active=[0, 1], width = 200)



theory_total= p.line('t','Total', source = source, color = 'red')
theory_total.visible = False    
    
args = dict(photo = photo, fp = fp, checkbox = checkbox_group)

for i, _ in enumerate(theory):
    args['theory_{}'.format(i)] = theory[i]

args['theory_total'] = theory_total



checkbox_update = CustomJS(args=args, code="""
    photo.visible = (checkbox.active.indexOf(0) > -1)
    fp.visible = (checkbox.active.indexOf(1) > -1)
    for( i= 0; i <= 15; ++i){
        eval("theory_" + i.toString()).visible =  (checkbox.active.indexOf(2) > -1)
    }
    theory_total.visible = (checkbox.active.indexOf(3) > -1)
""")
checkbox_group.js_on_click(checkbox_update)

background_button = Button(label = 'Subtract Background')
subtract_background = CustomJS(args= dict(source = source, button = background_button), code="""
    data = source.get('data')
    console.log('Hello World')
    if(button.label == 'Subtract Background'){
        data['photo'] = data['photo_nb']
        button.label = 'Add Background'
    }else{
        data['photo'] = data['photo_raw']
        button.label = 'Subtract Background'
    }
    source.change.emit()
    button.trigger('change')
""")
background_button.js_on_click(subtract_background)

invert_y_button = Button(label = 'Invert y-axis')
invert_y = CustomJS(args= dict(source = source, button = background_button), code="""
    data = source.get('data')
    for(i = 0; i< data['photo'].length; ++i){
        data['photo'][i] = -data['photo'][i]
    }
    source.change.emit()
""")
invert_y_button.js_on_click(invert_y)

paragraph = Paragraph(text="""Your text is initialized with the 'text' argument.  The
remaining Paragraph arguments are 'width' and 'height'. For this example, those values
are 200 and 100 respectively.""",
width=200, height=100)

paragraph2 = Paragraph(text="""Your text is initialized with the 'text' argument.  The
remaining Paragraph arguments are 'width' and 'height'. For this example, those values
are 200 and 100 respectively.""",
width=200, height=100)

In [151]:

xr  = (min(data['Saturated'].t)- 0.01 ,max(data['Saturated'].t)+0.01)
yr  =  (min(data['Saturated']['sat_background'])-0.001,max(data['Saturated']['sat_background'])+0.001)
p2 = figure(height = 300, width = 420, tools="pan,wheel_zoom,box_zoom,reset,box_select, lasso_select", 
            x_range = xr, y_range = yr, title = 'Spectrum with saturation beam')
p2.border_fill_color = 'lightgrey'


p2.yaxis.ticker = AdaptiveTicker()
p2.xaxis.axis_label = 'f[GHz]'
p2.yaxis.axis_label = 'Intensity (arb. units)'
p2.background_fill_color = 'black'
p2.grid.grid_line_alpha = .3

source_sat = ColumnDataSource(data=data['Saturated'])

photo_sat = p2.circle('t','photo',source = source_sat, size = 1.0, color = 'lightgreen')
photo_sat_line = p2.line('t','photo',source = source_sat,color = 'lightgreen')
photo_bckg = p2.line('t', 'sat_background', source = source_sat)
photo_link = p2.circle('t', 'link', source = source, alpha=1, size = 0)
photo_link2 = p2.circle('t', 'link2', source = source, alpha=1, size = 0)

smooth_button = Button(label = 'Smoothen')
smoothen = CustomJS(args= dict(source = source_sat, button = smooth_button), code="""
    data = source.get('data')
    console.log('Hello World')
    if(button.label == 'Smoothen'){
        data['photo'] = data['sat_filtered']
        button.label = 'Original'

    }else{
        data['photo'] = data['sat_nb']
        button.label = 'Smoothen'

    }
    source.change.emit()
    button.trigger('change')
    
""")
smooth_button.js_on_click(smoothen)
smooth_button.disabled = True

background_button2 = Button(label = 'Subtract Background')
subtract_background2 = CustomJS(args= dict(source = source_sat, button = background_button2, sb = smooth_button,
                                           photo_bckg = photo_bckg), code="""
    data = source.get('data')
    console.log('Hello World')
    if(button.label == 'Subtract Background'){
        data['photo'] = data['sat_nb']
        button.label = 'Add Background'
         photo_bckg.visible = false
        sb.disabled = false
    }else{
        data['photo'] = data['sat_raw']
        button.label = 'Subtract Background'
        photo_bckg.visible = true
        sb.disabled = true
        sb.label = 'Smoothen'
    }
    source.change.emit()
    button.trigger('change')
    
""")
background_button2.js_on_click(subtract_background2)



In [152]:
# dropdown.js_on_change('value', dropdown_update)
# show(layout([[widgetbox(checkbox_group, background_button, invert_y_button,paragraph),p],
#          [widgetbox(paragraph2),p2]]))
# lo = gridplot([[widgetbox(checkbox_group, background_button, invert_y_button,paragraph),p],
#                [widgetbox(paragraph2),p2]])
lo = gridplot([[ paragraph,p ,layout([[background_button], [invert_y_button],[checkbox_group]])],
               [paragraph2, p2,layout([[background_button2],[smooth_button]])]])
show(lo)
output_file('saturation_spectroscopy.html')
# save(row(widgetbox(checkbox_group),p))

In [128]:
data['Saturated'].head()

,t,sat_background,sat_raw,sat_nb,sat_filtered,photo
0,-0.533087,0.011181,0.010081,-0.0011,-0.000257,0.010081
1,-0.532298,0.011381,0.011081,-0.0003,-0.000257,0.011081
2,-0.529140,0.010381,0.011481,0.0011,-0.000259,0.011481
3,-0.526772,0.011181,0.011481,0.0003,-0.000256,0.011481
4,-0.524404,0.011181,0.010281,-0.0009,-0.000249,0.010281
